In [8]:
import struct
import numpy as np
from affine import Affine
from shapely.geometry import box
import geopandas as gpd


In [9]:
def read_geosoft_grid(file_path):
    """
    A hypothetical parser for a Geosoft Grid file.
    This example assumes the header contains:
      - width (int32)
      - height (int32)
      - minx (float64)
      - miny (float64)
      - maxx (float64)
      - maxy (float64)
    followed by width*height cells of float32 data.
    
    You will need to modify this based on the actual file format specification.
    """
    with open(file_path, 'rb') as f:
        # Read grid dimensions (assuming 4-byte integers)
        width_bytes = f.read(4)
        height_bytes = f.read(4)
        width = struct.unpack('i', width_bytes)[0]
        height = struct.unpack('i', height_bytes)[0]
        
        # Read spatial extent (assuming 8-byte float [double] values)
        minx = struct.unpack('d', f.read(8))[0]
        miny = struct.unpack('d', f.read(8))[0]
        maxx = struct.unpack('d', f.read(8))[0]
        maxy = struct.unpack('d', f.read(8))[0]
        
        # Create an affine transform from the extents and dimensions.
        # Here we assume the grid is organized with the upper-left corner at (minx, maxy).
        x_res = (maxx - minx) / width
        y_res = (maxy - miny) / height
        transform = Affine.translation(minx, maxy) * Affine.scale(x_res, -y_res)
        
        # Now read the grid values (assuming 4-byte float per cell)
        num_cells = width * height
        data = np.fromfile(f, dtype=np.float32, count=num_cells)
        data = data.reshape((height, width))
    
    return data, transform, width, height

def geosoft_grid_to_geoparquet(input_file, output_file):
    # Parse the geosoft grid file
    data, transform, width, height = read_geosoft_grid(input_file)
    features = []
    
    # Loop through each cell of the grid
    for row in range(height):
        for col in range(width):
            value = data[row, col]
            # Compute cell boundaries from the affine transform
            x_left, y_top = transform * (col, row)
            x_right, y_bottom = transform * (col+1, row+1)
            cell_polygon = box(x_left, y_bottom, x_right, y_top)
            features.append({'geometry': cell_polygon, 'value': value})
    
    # Define the CRS. Based on your metadata, the projection is "NAD83(CSRS) / Quebec Lambert" (EPSG:6622)
    crs = "EPSG:6622"
    
    # Create a GeoDataFrame and export it as a GeoParquet file.
    gdf = gpd.GeoDataFrame(features, crs=crs)
    gdf.to_parquet(output_file)
    print(f"Conversion complete. Output saved to {output_file}")


In [ ]:
# Example usage:
input_file = '/Users/thowe/MinersAI Dropbox/Science/Geo Data/Canada/Quebec/Geophysics/Magnetics/High Res/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.grd'   

output_file = '/Users/thowe/MinersAI Dropbox/Tyler Howe/QuebecProject/geo_data/output.parquet'  

geosoft_grid_to_geoparquet(input_file, output_file)